# 1. Genotype

First we load the eids of patients with a pathogenic variant in the Whole Exome 450K data

In [ ]:
outcome <- readRDS("outcome.rds")
head(outcome)

# 2. Phenotype

We then load the phenotype data

In [ ]:
require(data.table)
phenotype  <- as.data.frame(fread("data_participant.tsv"))
phenotype2 <- as.data.frame(fread("data2_participant.tsv"))
data <- subset(phenotype, eid %in% outcome$eid)
data2 <- subset(phenotype2, eid %in% outcome$eid)
data <- merge(data,data2, by="eid")

## 2.1 Prepare Phenotype

We create a new function to remove levels from factors

In [ ]:
require(dplyr)
removeLevels <- function(data, variable, excl.values){
  data[which(data[,variable] %in% excl.values),variable]<- NA
  data[,variable] <- droplevels(data[,variable])
}

### 2.1.1 Demographics

#### 2.1.2  Gender

In [ ]:
data$gender <- data$p31
data$gender <- as.factor(data$gender)
table(data$gender) # need to recode as in FHSC

#### 2.1.3 Country

In [ ]:
data$country <- data$p1647_i0
data$country[data$country==""] <- "Unknown"
data$country <- as.factor(data$country )

data$country <- recode_factor(data$country,
                              'Republic of Ireland'="Elsewhere", 
                              'Do not know'='unknown/missing',
                              'Prefer not to answer'='unknown/missing')

data$country <- removeLevels(data, "country", c("unknown/missing","Unknown")) 
table(data$country)
#unique(data$country)  # not very useful either UK or elsewhere!

#### 2.1.4 Ethnicity

In [ ]:
data$ethnicity <- data$p21000_i0
data$ethnicity[data$ethnicity==""] <- "unknown"
data$ethnicity <- as.factor(data$ethnicity )

data$ethnicity <- recode_factor(data$ethnicity,
                                  'Do not know'='unknown',
                                  'Prefer not to answer'='unknown/missing',
                                  "Other ethnic group"="unknown/missing",
                                  "African"="black",
                                  "Caribbean"="black",
                                  "Black or Black British"="black",
                                  "Any other Black background"="black",
                                  "British"="white",
                                  "White"="white",
                                  "Any other white background"="white",
                                  "Irish"="white",
                                  "White and Black Caribbean"="mixed",
                                  "White and Black African"="mixed",
                                  "Any other mixed background"="mixed",
                                  "Mixed"="mixed",
                                  "White and Asian"="mixed",
                                  "Chinese"="oriental/Asian",
                                  "Indian"="oriental/Asian",
                                  "Pakistani"="oriental/Asian",
                                  "Bangladeshi"="oriental/Asian",
                                  "Asian or Asian British"="oriental/Asian",
                                  "Any other Asian background"="oriental/Asian")
data$ethnicity <- removeLevels(data, "ethnicity", c("unknown/missing","unknown")) 

table(data$ethnicity) # useful 

#### 2.1.5  Occupation

In [ ]:
occupation_mapping <- read.csv("tables/occupations_table.csv")
data$occupation<- data$p20277_i0
data$occupation[data$occupation==""]<-"unknown/missing"
data$occupation <- sapply(data$occupation, FUN =function(x){gsub("-", "", x)} )
for(i in 1:nrow(occupation_mapping)){
    data$occupation[data$occupation==occupation_mapping[i,1]]<-occupation_mapping[i,2]
}
data$occupation <- as.factor(data$occupation)
data$occupation <- removeLevels(data, "occupation", c("unknown/missing")) 
table(data$occupation)

#### 2.1.6 Education

In [ ]:
data$education <- data$p6138_i0
# find Highest qualification
qualifications <- c("College or University degree", "A levels/AS levels or equivalent",
                    "O levels/GCSEs or equivalent","CSEs or equivalent","NVQ or HND or HNC or equivalent",
                    "Other professional qualifications eg: nursing, teaching","None of the above","Prefer not to answer")
data$education_tmp <- "unknown/missing"
for( i in length(qualifications):1){
    data$education_tmp <- ifelse(grepl(qualifications[i],data$education),qualifications[i],data$education_tmp)
}
data$education <- as.factor(data$education_tmp)
data$education <- removeLevels(data, "education", c("unknown/missing")) 

table(data$education)
#sum(!is.na(data$education))
#unique(data$education) # -- too many occupation need to regroup

#### 2.1.7 Year of birth (DOB)

In [ ]:
data$dob <- as.numeric(data$p34)
hist(data$dob )

#### 2.1.8  Age at entry

In [ ]:
data$age <- data$p21003_i0
hist(data$age)

### 2.2 Anthrompometric Measurements

#### 2.2.1 Waist Circumference

In [ ]:
data$waist <- data$p48_i0
hist(data$waist)

#### 2.2.2 Weight

In [ ]:
data$weight <- data$p21002_i0
hist(data$weight)

#### 2.2.3 BMI

In [ ]:
# max of boy mass index 
data$bmi <- apply( data[,paste('p21001_i',0:3,sep="")],1, FUN = function(x){
  x <- na.omit(x)
  if (length(x)>0){max(x)}else{return(NA)}
})
data$bmi_cat <- ifelse( data$bmi < 18.5, "underweight",ifelse( data$bmi < 25, "normal", ifelse( data$bmi < 30, "overweight", "obese")))
hist(data$bmi)
table(data$bmi_cat)

#### 2.2.4 Height

In [ ]:
data$height <- sqrt(data$weight/data$bmi)
hist(data$height)

#### 2.2.5 SBP

In [ ]:
columns <- as.vector(sapply(paste('p4080_i',0:2,"_a",sep=""), paste, c("0","1"), sep=""))
data$sbp <- apply( data[,columns],1, FUN = function(x){
  x <- na.omit(x)
  if (length(x)>0){max(x)}else{return(NA)}
})
hist(data$sbp)

#### 2.2.6 DBP

In [ ]:
columns <- as.vector(sapply(paste('p4079_i',0:2,"_a",sep=""), paste, c("0","1"), sep=""))
data$dbp <- apply( data[,columns],1, FUN = function(x){
  x <- na.omit(x)
  if (length(x)>0){max(x)}else{return(NA)}
})
hist(data$dbp)

#### 2.2.7 HR

In [ ]:
columns <- as.vector(sapply(paste('p102_i',0:2,"_a",sep=""), paste, c("0","1"), sep=""))
data$hr <- apply( data[,columns],1, FUN = function(x){
  x <- na.omit(x)
  if (length(x)>0){max(x)}else{return(NA)}
})
hist(data$hr)

### 2.3 Lifestyle
#### 2.3.1  Alcohol frequency

In [ ]:
data$alcohol <- as.factor(data[,"p1558_i0"])
data$alcohol <- removeLevels(data, "alcohol", c("")) 
table(data$alcohol)

#### 2.3.2 Smoking status

In [ ]:
data$smoking <- as.factor(data[,"p20116_i0"])
data$smoking <- removeLevels(data, "smoking", c("")) 
table(data$smoking )

#### 2.3.3 Smoking Pack-year

In [ ]:
data$smoking_py <- as.numeric(data[,"p20161_i0"])
hist(data$smoking_py )

#### 2.3.4 Smoking time

In [ ]:
options(warn=-1)
data$smoking_start <- apply(data[,paste("p3436_i",0:3, sep="")], 1 , FUN= function(x) {
    x[x==""] <- NA
    x <- na.omit(x)
    if(length(x)==0){
        return(NA)
    }else{
        return(min(as.numeric(x)))
    }     
})
data$smoking_quit <- apply(data[,paste("p6194_i",0:3, sep="")], 1 , FUN= function(x) {
    x[x==""] <- NA
    x <- na.omit(x)
    if(length(x)==0){
        return(NA)
    }else{
        return(max(as.numeric(x)))
    }     
})
options(warn=1)

data$smoking_time <-  as.numeric(data$smoking_quit-data$smoking_start)
hist(data$smoking_time)

#### 2.3.5  Physical Activity 

In [ ]:
data$activity_computer <- as.numeric(data[,'p1080_i0'] )
data$activity_tv       <- as.numeric(data[,'p1070_i0'] )
hist(data$activity_computer)
hist(data$activity_tv)

### 2.3.6 Diet

In [ ]:
data$diet_meat         <- data[,'p1349_i0']
data$diet_oily_fish    <- data[,'p1329_i0']
data$diet_nonoily_fish <- data[,'p1339_i0']
data$diet_cooked_veg   <- data[,'p1289_i0']
data$diet_raw_veg      <- data[,'p1299_i0']

### 2.4 Comorbidities

#### 2.4.1  Comorbidities (ICD10 codes)

The code below load the data.frame containing the set of ICD10 code for each co-morbidities variables.
For each patient and each variable, it then looks if there is a matching ICD10 code. 
If there is the age columns at onset are retrieved from another set of UK Biobank fields.     

In [ ]:
file_icd10 <- read.csv("tables/morbidities_final.csv")
### Find if ECD10 coded and earliest age 
getEarliestEventAndDate <- function(data_input, icd10_codes){  
    ## provide ICD10
    icd10_codes_regex <- paste("^.*(", gsub(".","\\.",paste(icd10_codes,collapse="|"), fixed=T), ").*$",sep="")
    ## Find which one match
    data_input$has_event <- apply( data_input[,c('p41270','p41202','p41204')], 1, FUN = function(x){
        newX <- paste(x, collapse= " |--> ")
        return(ifelse(grepl(icd10_codes_regex, newX),1,0))
    })
    data_input[ data_input$has_event==1, "age_earliest_event"] <- apply(data_input[ data_input$has_event==1,c('p41270','p41202',
                        "dob",
                        paste("p41280_a",c(0:180), sep=""),
                        paste("p41262_a",c(0:74), sep=""))],1, FUN=function(x){ 
                                vect1 <- strsplit(as.character(x[1]), "\\|")
                                datesFields1 <- x[4:(181+4)]
                                dates1 <- datesFields1[unlist(sapply(vect1, FUN = function(y) { grep(icd10_codes_regex,y) }))]
                                dx_dates1 <- unlist(sapply(dates1, as.Date, origin="1970-01-01"))

                                vect2 <- strsplit(as.character(x[2]), "\\|")
                                datesFields2 <- x[(181+5):length(x)]
                                dates2 <- datesFields2[unlist(sapply(vect2, FUN = function(y2) { grep(icd10_codes_regex,y2) }))]
                                dx_dates2 <- unlist(sapply(dates2, as.Date, origin="1970-01-01"))

                                allDates <- na.omit(c(dx_dates1,dx_dates2))
                                if(length(allDates)>0){
                                    minDate <- min(allDates)
                                    if (is.null(minDate) | is.na(minDate) | is.infinite(minDate)){
                                        return(NA)
                                    }else{
                                        return(as.numeric(format(as.Date(minDate, origin="1970-01-01"),"%Y")) - as.numeric(x[3]) )
                                    }
                                }else{
                                  return(NA)   
                                }
    })
    return(data_input[,c("has_event","age_earliest_event")])
}

In [ ]:
for ( disease in colnames(file_icd10)[! colnames(file_icd10) %in% c("Code", "Disease")]) {
    message(paste("starting ", disease, sep=""))
    icd10_filtered                    <- getEarliestEventAndDate(data, file_icd10[file_icd10[,disease]==1,"Code"])
    data[, paste(disease, "_dx",sep="")]     <- icd10_filtered$has_event
    data[, paste(disease, "_dx",sep="")]     <- recode_factor(data[, paste(disease, "_dx",sep="")], "0"="no","1"="yes")
    data[, paste(disease, "_age",sep="")]    <- icd10_filtered $age_earliest_event
    message(paste(disease, " ready",sep=""))
}

### 2.5 Operations

#### 2.5.1  Operations (OPSC4 codes)

In [ ]:
file_opsc4 <- read.csv("tables/operations_final.csv")
getOPSC4   <- function(data_input, opsc4_codes){
    opsc4_codes_regex <- paste("^.*(", paste(opsc4_codes,collapse="|"), ").*$",sep="")
    ## Find which one match
    return(apply( data_input[,c('p41272','p41200','p41210')], 1, FUN = function(x){
        newX <- paste(x, collapse= " |--> ")
        return(ifelse(grepl(opsc4_codes_regex, newX),1,0))
    }))
}

In [ ]:
for ( operation in colnames(file_opsc4)[! colnames(file_opsc4) %in% c("Full.Name", "Code")][2:5]) {
    message(paste("starting ", operation, sep=""))
    data[, operation ]    <- getOPSC4(data, file_opsc4[file_opsc4[,operation]==1,"Code"])
    data[, operation ]    <- recode_factor(data[, operation ], "0"="no","1"="yes")
    message(paste(operation, " ready",sep=""))
}

### 2.6 Imaging test

#### 2.6.1 Carotid IMT

In [ ]:
data$us_carotid_left_img1 <- data[,c("p20222_i2")]
data$us_carotid_left_img2 <- data[,c("p20222_i3")]

data$us_carotid_right_img1 <- data[,c("p20223_i2")]
data$us_carotid_right_img2 <- data[,c("p20223_i3")]

In [ ]:
maxIMT <- function(x) { 
    x <- as.numeric(x)
    if(sum(is.na(x)) == 2 )
        return(NA)
    else
        return (max(x, na.rm=T)) 
}
options(warn=-1)
data$uscar_imt_120_max <- apply(data[,c("p22671_i2","p22671_i3")], 1, FUN= maxIMT)
data$uscar_imt_150_max <- apply(data[,c("p22674_i2","p22674_i3")], 1, FUN= maxIMT)
data$uscar_imt_210_max <- apply(data[,c("p22677_i2","p22677_i3")], 1, FUN= maxIMT)
data$uscar_imt_240_max <- apply(data[,c("p22680_i2","p22680_i3")], 1, FUN= maxIMT)
options(warn=1)

#### 2.6.2  Arterial Stifness Index

In [ ]:
maxIMT <- function(x) { 
    x <- as.numeric(x)
    if(sum(is.na(x)) == 4 )
        return(NA)
    else
        return (max(x, na.rm=T)) 
}

data$max_art_stiffness <- apply(data[,c("p21021_i0", "p21021_i1", "p21021_i2", "p21021_i3")], 1, FUN = maxIMT)

#### 2.6.3  ECG

In [ ]:
data$ecg_dataset1 <- data[, 'p20205_i2'] 
data$ecg_dataset2 <- data[, 'p20205_i3'] 

#### 2.6.4 MRI

In [ ]:
data$mri_scout_img1 <- data[,'p20207_i2']
data$mri_scout_img2 <- data[,'p20207_i3']

data$mri_short_axis_img1 <- data[,'p20209_i2' ]
data$mri_short_axis_img2 <- data[,'p20209_i3']

data$mri_lv_out_img1 <- data[,'p20212_i3' ]
data$mri_lv_out_img2 <- data[,'p20212_i2']

data$mri_long_axis_img1 <- data[,'p20208_i2']
data$mri_long_axis_img2 <- data[,'p20208_i3' ]

### 2.7 Family History

In [ ]:
# These seem a little off! 
relevant_columns <- c(paste("p20107_i",0:3, sep=""),
                      paste("p20110_i",0:3, sep=""),
                      paste("p20111_i",0:3, sep=""))

#### 2.7.1 Family history of ischaemic heart disease and other diseases of the circulatory system

In [ ]:
data$famh_hd_sa <- apply( data[,relevant_columns], 1, FUN = function(x){
    ifelse(sum(grepl("Heart disease", x))>=1,1,0)
})
data$famh_hd_sa    <- recode_factor(data$famh_hd_sa, "0"="no","1"="yes")
table(data$famh_hd_sa)

icd10_fmh_cad <- getEarliestEventAndDate(data, c(paste("Z", c("82.4"), sep="")))
data$famh_cad <- icd10_fmh_cad$has_event
data$famh_cad <- recode_factor(data$famh_cad, "0"="no","1"="yes")
table(data$famh_cad)

#### 2.7.2 Family history of hypercholesterolemia

In [ ]:
icd10_fmh_cad <- getEarliestEventAndDate(data, c(paste("Z", c("83\\.43"), sep="")))
sum(icd10_fmh_cad$has_event==1)
# no patient found with this code!!!

#### 2.7.2 Family history of stroke

In [ ]:
data$famh_stroke_sa <- apply( data[,relevant_columns], 1, FUN = function(x){
    ifelse(sum(grepl("Stroke", x))>=1,1,0)
})
data$famh_stroke_sa <- recode_factor(data$famh_stroke_sa, "0"="no","1"="yes")
table(data$famh_stroke_sa)

icd10_fmh_stroke <- getEarliestEventAndDate(data, c(paste("Z", c("82.3"), sep="")))
data$famh_stroke <- icd10_fmh_stroke$has_event
data$famh_stroke <- recode_factor(data$famh_stroke, "0"="no","1"="yes")
table(data$famh_stroke)

#### 2.7.3 Family history of Diabetes Melitus

In [ ]:
data$famh_dm_sa <- apply( data[,relevant_columns], 1, FUN = function(x){
    ifelse(sum(grepl("Diabetes", x))>=1,1,0)
})
data$famh_dm_sa <- recode_factor(data$famh_dm_sa, "0"="no","1"="yes")
table(data$famh_dm_sa)

icd10_fmh_dm <- getEarliestEventAndDate(data, c(paste("Z", c("83.3"), sep="")))
data$famh_dm <- icd10_fmh_dm$has_event
data$famh_dm <- recode_factor(data$famh_dm, "0"="no","1"="yes")
table(data$famh_dm)

#### 2.7.4 High blood pressure

In [ ]:
data$famh_ht_sa <- apply( data[,relevant_columns], 1, FUN = function(x){
    ifelse(sum(grepl("High blood pressure", x))>=1,1,0)
})
data$famh_ht_sa <- recode_factor(data$famh_ht_sa, "0"="no","1"="yes")

table(data$famh_ht_sa)

#### 2.7.5 Cancer

In [ ]:
data$famh_cancer_sa <- apply( data[,relevant_columns], 1, FUN = function(x){
    ifelse(sum(grepl("cancer", x))>=1,1,0)
})
data$famh_cancer_sa <- recode_factor(data$famh_cancer_sa, "0"="no","1"="yes")
table(data$famh_cancer_sa)

icd10_fmh_neoplasm <- getEarliestEventAndDate(data, c(paste("Z", c("80"), sep="")))
data$famh_cancer   <- icd10_fmh_neoplasm$has_event
data$famh_cancer   <- recode_factor(data$famh_cancer, "0"="no","1"="yes")
table(data$famh_cancer)

#### 2.7.6 Overall  Death non-accidental

In [ ]:
relevant_columns <- paste('p4501_i',0:3, sep="")
data$famh_death_sa <- apply( data[,relevant_columns], 1, FUN = function(x){
    ifelse(sum(grepl("Yes", x))>=1,1,0)
})
data$famh_death_sa <- recode_factor(data$famh_death_sa, "0"="no","1"="yes")
table(data$famh_death_sa)

#### 2.8 Lab measurements

In [ ]:
labs <- read.csv("tables/labs.csv")

In [ ]:
for(i in 1:nrow(labs)){
    data[,labs[i,"field"]] <- apply(data[,paste(labs[i,"code"],0:1, sep="")], 1 , FUN= function(x) {
    x[x==""] <- NA
    x <- na.omit(x)
    if(length(x)==0){
        return(NA)
    }else{
        return(max(as.numeric(x)))
    }     
})
}

In [ ]:
data$ldl_fw  <- (data %>% mutate(ldl_fw = tc-hdl-tg/2.2))$ldl_fw
data$nonhdl  <- data$tc - data$hdl
data$lpa_cat <- as.factor(ifelse( data$lpa > 105, 1, 0 ) )
table(data$lpa_cat )

### 2.8 Medications

In [ ]:
meds <- read.csv("tables/medications_final.csv")

In [ ]:
columns <- paste("p20003_i",0:3, sep="")
getMedications <- function(data, med_names){
    apply( data, 1, FUN = function(x){
        x[x==""]<-NA
        x <- na.omit(x)
        collapsed <- paste(x, collapse="|")
        vect2 <- unlist(strsplit(collapsed, "\\|"))
        ifelse(any(vect2 %in% med_names),1,0)
    })
}

In [ ]:
for ( med_name in colnames(meds)[! colnames(meds) %in% c("Category","Count","ATC")]) {
    data[,med_name] <- getMedications(data[,columns],meds[meds[,med_name]!="","Category"])
    data[,med_name] <- as.factor(data[,med_name])
}

## 3. Finalise And Save Dataset

We merge the phenotype dataset with the outcome and save the dataset containing the new columns 

In [ ]:
prepared_dataset <- merge(data, outcome, by="eid")
saveRDS(prepared_dataset, "prepared_dataset.rds")

In [ ]:
colnames(prepared_dataset)

# 4. Feature Engineering 

## 4.1 Load dataset

In [ ]:
prepared_dataset <- readRDS("prepared_dataset.rds")

## 4.2 Remove original columns from UK Biobank and "image" columns (we do not need them for this analysis)

In [ ]:
cols_remove <- colnames(prepared_dataset)[grep("^((p[0-9]{1,7}(_[ai0-9]{0,10}){0,1}(_[ai0-9]{0,10}){0,1})|(.*img.*)|(.*dataset.*))$", colnames(prepared_dataset)) ]
prepared_dataset[ , cols_remove ] <- NULL
colnames(prepared_dataset)

## 4.3 Derive the "disease" columns

In [ ]:
cols_disease <- colnames(prepared_dataset)[grep("^.*_dx$", colnames(prepared_dataset)) ]
cols_age     <- gsub("_dx","_age", cols_disease)
cols         <- gsub("_dx","", cols_disease)
df           <- data.frame(dx = cols_disease, age = cols_age, col = cols )
gender       <-  prepared_dataset[, "gender"]
for (cdisease in 1:nrow(df)){
    disease <- prepared_dataset[, df[cdisease, "dx"]]
    onset   <- prepared_dataset[, df[cdisease, "age"]]
    prepared_dataset[, df[cdisease, "col"]]  <- ifelse(   disease == "no", 
                                                           "no disease", 
                                                           ifelse(disease == "yes",  
                                                                  ifelse(gender == "Female",  
                                                                         ifelse(onset < 60, 
                                                                                "premature disease", 
                                                                                ifelse(onset >= 60, 
                                                                                        "disease non-premature",
                                                                                        "disease onset unknown")),
                                                                          ifelse(gender == "Male",   
                                                                                ifelse(onset < 55, 
                                                                                        "premature disease", 
                                                                                        ifelse(onset >= 55, 
                                                                                               "disease non-premature",
                                                                                               "disease onset unknown")),
                                                                                "disease onset unknown")),
                                                                    NA))
    prepared_dataset[, df[cdisease, "col"]] <- as.factor(prepared_dataset[, df[cdisease, "col"]] )
}

 Check values

In [ ]:
require(dplyr)
subset(prepared_dataset[, c("gender", "diabetes2_dx","diabetes2","diabetes2_age")],diabetes2 == "premature disease") %>% group_by(gender) %>% summarise(minage=min(diabetes2_age),maxage=max(diabetes2_age))

Remove intermediate variables

In [ ]:
prepared_dataset$education_tmp <- NULL
prepared_dataset[, grepl("_dx", colnames(prepared_dataset))] <- NULL
prepared_dataset[, grepl("_age", colnames(prepared_dataset))] <- NULL
prepared_dataset[,"eid"] <- NULL
#colnames(prepared_dataset)

## 4.4 Remove column with more than 20% missing

In [ ]:
missing_columns <- apply(prepared_dataset, 2, function(x) ( sum(is.na(x))/length(x)> 0.20 ) )
cols_remove      <- colnames(prepared_dataset)[missing_columns]
prepared_dataset[ , cols_remove ] <- NULL      
colnames(prepared_dataset)

## 4.5 Set correct types for variables

In [ ]:
prepared_dataset$bmi_cat <- as.factor(prepared_dataset$bmi_cat)
prepared_dataset$age_cat <- as.factor(prepared_dataset$age_cat)
prepared_dataset$diet_meat <- as.factor(prepared_dataset$diet_meat)
prepared_dataset$diet_oily_fish <- as.factor(prepared_dataset$diet_oily_fish)
prepared_dataset$diet_nonoily_fish <- as.factor(prepared_dataset$diet_nonoily_fish)
prepared_dataset$diet_cooked_veg <- as.numeric(prepared_dataset$diet_cooked_veg)
prepared_dataset$diet_raw_veg <- as.numeric(prepared_dataset$diet_raw_veg)
prepared_dataset$ANY <- as.factor(prepared_dataset$ANY)
prepared_dataset$APOB <- as.factor(prepared_dataset$APOB)
prepared_dataset$PCSK9 <- as.factor(prepared_dataset$PCSK9)
prepared_dataset$LDLR <- as.factor(prepared_dataset$LDLR)
prepared_dataset$bmi_cat <- as.factor(prepared_dataset$bmi_cat)

## 4.6 Replace "Prefer not to answer" with null (to decrease the number of rare levels)

The motivation for doing so is that we do not want some of our dataset (feature importance/derivation/validation) with different levels of factors. 
Sinc the overall dataset is 500,000 and we have 10 dataset we want to keep levels at least 10 times in the dataset. 

In [ ]:
col_factors <- colnames(prepared_dataset)[unlist(lapply(prepared_dataset, is.factor))]
for (c in col_factors){
    if ( "Prefer not to answer" %in% levels(prepared_dataset[,c])){
        prepared_dataset[ !is.na(prepared_dataset[,c]) & prepared_dataset[,c]=="Prefer not to answer", c] <- NA
    }
}
prepared_dataset <- droplevels(prepared_dataset)

## 4.7 Remove variable with rare levels < 1:50,000; 


In [ ]:
for (c in col_factors){
    tbl <- table(prepared_dataset[,c])/nrow(prepared_dataset)
    if(length(tbl[tbl<0.00002])!=0){
        cat("\t",c, "\n")
        cat(names(tbl[tbl<0.00002]),"\n")
    }
}

In [ ]:
prepared_dataset$PCSK9             <- NULL
prepared_dataset$APOB             <- NULL
# remove unused levels
prepared_dataset <- droplevels(prepared_dataset)

# 5. Perform Data Partitioning

## 5.1 Perform stratification by age (median), sex and FH-carrying status

In [ ]:
prepared_dataset$age_cat <- ifelse(prepared_dataset$age < 58, 0, 1)
strata <-  prepared_dataset %>% group_by(gender, age_cat,ANY) %>% summarise(count= n())
strata 

In each strata, for each patient randomly allocated a number between 1 and 10.  

In [ ]:
set.seed(123)

attributeFold <- function(data, k){
    data <- data[sample(1:nrow(data)), ] # shuffling
    eachGrp  <- floor(nrow(data)/k)      # find number of individual in each group
    reminder <- nrow(data)-eachGrp*k     # find how many cannot fit in a group 
    return(c(rep(1:k, eachGrp), sample(1:k, reminder,replace=F)))
}

k <- 10 
datasetList <- list()
for (i in 1:nrow(strata)){
    sdataset <- subset(prepared_dataset, gender ==  strata[i, "gender"]$gender & age_cat == strata[i, "age_cat"]$age_cat  & ANY == strata[i, "ANY"]$ANY)
    sdataset$fold <- attributeFold(sdataset, k)
    datasetList[[ length( datasetList ) + 1 ]] <- sdataset
}
sum(unlist(lapply(datasetList, nrow)))

Pull all the data frame together

In [ ]:
merged_data <- do.call(rbind, datasetList)
nrow(merged_data )

In [ ]:
merged_data  %>% group_by(fold) %>% summarise(gender = sum(ifelse(gender=="Male",1,0)), age_cat = sum(age_cat),ANY = sum(ANY=="1"))

## 5.2 Attribute subset of dataset by random/stratified folds to the  3 datasets

In [ ]:
feature_importance_dataset <- subset(merged_data, fold %in% 1:2 )
derivation_dataset         <- subset(merged_data, fold %in% 3:8)
validation_dataset         <- subset(merged_data, fold %in% 9:10)

feature_importance_dataset$fold <- NULL
derivation_dataset$fold         <- NULL
validation_dataset$fold         <- NULL

### 5.2.1 Furter stratify the derivation dataset into 5 folds using (median age, sex, FH-carrying status)

In the derivation dataset we want to assign 5 folds for CV. 

In [ ]:
strata <-  derivation_dataset %>% group_by(gender, age_cat,ANY) %>% summarise(count= n())
strata 

In [ ]:
nrow(derivation_dataset)
k <- 5 
datasetList <- list()
for (i in 1:nrow(strata)){
    sdataset <- subset(derivation_dataset, gender ==  strata[i, "gender"]$gender & age_cat == strata[i, "age_cat"]$age_cat  & ANY == strata[i, "ANY"]$ANY)
    sdataset$fold <- attributeFold(sdataset, k)
    datasetList[[ length( datasetList ) + 1 ]] <- sdataset
}
derivation_dataset <- do.call(rbind, datasetList)
nrow( derivation_dataset)
derivation_dataset  %>% group_by(fold) %>% summarise(gender = sum(ifelse(gender=="Male",1,0)), age_cat = sum(age_cat),ANY = sum(ANY=="1"))

We now want to save our work to the corresponding file. 

## 5.3 Save the stratified datasets

In [ ]:
saveRDS( feature_importance_dataset, "./DataPartition/feature_importance_dataset.RDS")
saveRDS( derivation_dataset        , "./DataPartition/derivation_dataset.RDS")
saveRDS( validation_dataset        , "./DataPartition/validation_dataset.RDS")

# 6. Perform Missing Data Imputation

## 6.1 Load the data

In [ ]:
feature_importance_dataset <- readRDS( "./DataPartition/feature_importance_dataset.RDS")
derivation_dataset         <- readRDS( "./DataPartition/derivation_dataset.RDS")
validation_dataset         <- readRDS( "./DataPartition/validation_dataset.RDS")

## 6.2 Install/Load the packages

In [ ]:
# install.packages("mice")
# install.packages("future")
# install.packages("furrr")
# install.packages("nnet")
require(mice) # install.packages("mice")
require(furrr) # install.packages("furrr")

## 6.3 Impute Feature Importance Dataset

In [ ]:
mids <-  futuremice(data = feature_importance_dataset,
                  m = 5, parallelseed = 123, use.logical = FALSE,
                  n.core = 5,  maxit = 1)
feature_importance_imputed_dataset <- complete(mids)
sum(is.na(feature_importance_imputed_dataset))

In [ ]:
saveRDS( feature_importance_imputed_dataset      , "./DataPartition/feature_importance_imputed_dataset.RDS")

## 6.4 Impute Derivation dataset by CV fold

In [ ]:
listImpDataset <- list()
for (k in 1:max(derivation_dataset$fold)) {
    message(paste("k is ", k, sep=""))
    sdataset    <- subset(derivation_dataset, fold == k)
    mids        <-  futuremice(data = sdataset, use.logical = FALSE, 
                  m = 5, parallelseed = 123, 
                  n.core = 5,  maxit = 1)
    listImpDataset[[ length( listImpDataset ) + 1  ]] <- complete(mids)
}
derivation_dataset_imputed_dataset <- do.call(rbind, listImpDataset)

In [ ]:
saveRDS( derivation_dataset_imputed_dataset        , "./DataPartition/derivation_imputed_dataset.RDS")

## 6.5 Impute Validation dataset

In [ ]:
mids <-  futuremice(data = validation_dataset,
                  m = 5, parallelseed = 123, use.logical = FALSE, 
                  n.core = 5,  maxit = 1)
validation_imputed_dataset <- complete(mids)

In [ ]:
saveRDS( validation_imputed_dataset                , "./DataPartition/validation_imputed_dataset.RDS")

In [ ]:
numeric_columns <- lapply(feature_importance_imputed_dataset, FUN = is.numeric) 

# Scaling variables

In [ ]:
feature_importance_imputed_scaled_dataset      <- readRDS("./DataPartition/feature_importance_imputed_dataset.RDS")
derivation_imputed_scaled_dataset              <- readRDS("./DataPartition/derivation_imputed_dataset.RDS")
validation_imputed_scaled_dataset              <- readRDS("./DataPartition/validation_imputed_dataset.RDS")

In [ ]:
feature_importance_imputed_scaled_dataset$pcsk9 <- NULL
derivation_imputed_scaled_dataset$pcsk9         <- NULL
validation_imputed_scaled_dataset$pcsk9         <- NULL

In [ ]:
feature_importance_imputed_scaled_dataset$age_cat <- as.factor(feature_importance_imputed_scaled_dataset$age_cat)
derivation_imputed_scaled_dataset$age_cat         <- as.factor(derivation_imputed_scaled_dataset$age_cat)
validation_imputed_scaled_dataset$age_cat         <- as.factor(validation_imputed_scaled_dataset$age_cat)

In [ ]:
numeric_columns <- colnames(feature_importance_imputed_scaled_dataset)[ unlist(lapply(feature_importance_imputed_scaled_dataset, FUN = is.numeric))]
cat_columns     <- colnames(feature_importance_imputed_scaled_dataset)[ unlist(lapply(feature_importance_imputed_scaled_dataset, FUN = is.factor))]

In [ ]:
feature_importance_imputed_scaled_dataset[,numeric_columns] <- scale(feature_importance_imputed_scaled_dataset[,numeric_columns])
derivation_imputed_scaled_dataset[,numeric_columns]         <- scale(derivation_imputed_scaled_dataset[,numeric_columns])
validation_imputed_scaled_dataset[,numeric_columns]         <- scale(validation_imputed_scaled_dataset[,numeric_columns])

In [ ]:
saveRDS(feature_importance_imputed_scaled_dataset, "./DataPartition/feature_importance_imputed_scaled_dataset.RDS")
saveRDS(derivation_imputed_scaled_dataset, "./DataPartition/derivation_imputed_scaled_dataset.RDS")
saveRDS(validation_imputed_scaled_dataset, "./DataPartition/validation_imputed_scaled_dataset.RDS")